# **Ejercicio 1.1**
**Análisis Exploratorio de Datos.**  
Dado los siguientes conjuntos de datos: **Default Probability Prediction** y **Wind Speed**, realizar un análisis exploratorio de datos el cual incluya lo siguiente:

- **Descripción de tipos de variables**, **reducción de nombres extensos en columnas**, calcular **número de observaciones**, **media**, **desviación estándar**, **mínimo**, **máximo**, **cuartiles**, realizar **conteo de datos faltantes y su porcentaje**, **histograma** o **diagrama de barras** para la variable respuesta e independientes según corresponda. Seleccionar un mínimo de 10 variables independientes, para este análisis. **Análisis de simetría**, **datos atípicos** y **dispersión**, etc., por medio de `boxplot()`.  **Análisis bivariado**. Trazado de `scatterplot()` y `regplot()` para un mínimo de 10 pares de variables explicativas. En cada figura agregar un análisis y descripción.

- Según corresponda, realizar **imputación de datos faltantes** por medio de **Imputación Iterativa Múltiple**. Realizar **reducción de dimensionalidad** por medio de eliminación de columnas altamente correlacionadas usando **Variance Inflation Factor (VIF)**. Para esto se recomienda usar la siguiente librería `variance_inflation_factor()`. Un **VIF ≥ 10** indica alta multicolinealidad entre la correspondiente variable independiente y las demás variables. **Recomendación**: Eliminar una columna a la vez. Aquella con el máximo **VIF ≥ 10**. Luego, para el nuevo `pandas`, calcular nuevamente **VIF** e identificar nuevas columnas con **VIF ≥ 10** máximo, y así sucesivamente hasta obtener solo valores de **VIF < 10**.  Según corresponda, variables categóricas deben previamente codificarse usando por ejemplo `OneHotEncoder()`.  Pueden mantener las variables categóricas antes de la codificación previa al entrenamiento del modelo y **reducir multicolinealidad** usando la prueba `chi2_contingency()`. Como **Científico de Datos**, queda a su juicio la decisión sobre qué variables independientes deberían ser eliminadas. Justifique en cada caso su respuesta. Puede proponer una metodología alternativa de reducción de dimensionalidad que considere más pertinente. En cada caso, justifique su respuesta. Puede evitar eliminar variables en el dataset de **Wind Speed**, debido a la baja cantidad de características.
